# 모델의 학습과 최적화 이론

## 지도 학습 vs 비지도 학습
 - 지도 학습(Supervised Learning) : 입력과 함께 정답을 알려주고 그 정답을 맞추도록 하는 학습 방법
 - 비지도 학습(Unsupervised Learning) : 정답의 제공 없이 데이터로부터 유용한 정보를 추출하는 학습 방법

## 학습 매개변수(Trainable Parameter)
- 학습 과정에서 값이 변화하는 매개 변수, 이 ㄱ밧이 변화하면 알고리즘 출력이 변화

## 손실 함수(Loss Function)
- 알고리즘이 얼마나 잘못 하고 있는지를 표현하는 지표. 값이 낮을 수록 학습이 잘 된 것
- 정답과 알고리즘 출력을 비교하는 데에 사용한다.
- MSE, CEE

## 학습 환경의 정의
 - Training Data | Model(Trainable Parameters) | Loss Function
 - 알고리즘 학습 -> "입력을 바꿔가면서, 출력 값이 점점 작아지게 한다"
 - 모델을 학습 한다는 것 = 최적화 이론 = 출력 값을 가장 작게하는 입력 값을 찾는다.
   - y = ax + b , y:출력, a, b:입력

## 경사 하강 학습법 (최적화 이론 중 하나)
 - 최적화 이론 : 손실 함수를 최소로 하는 입력 값(최적 값)을 찾는 것
 - ex)무차별 대입법 Brute-Force : 가능한 모든 수를 넣어보는 단순값
   - 최적값 범위를 알아야 하고 계산 복잡도가 높으며 무한히 촘촘하게 조사해야 함
   - 적대 대입해 보고 답을 찾을 수는 없을까?
 - 경사 하강법(Gradient Descent) : 경사를 따라 여러 번의 스텝을 통해 최적점으로 다가간다. 경사는 기울기(미분, Gradient)를 이용해 계산한다.
   - 학습률 : 학습률(a, alpha)에 비례하여 이동한다. 너무 크면 진동하고, 너무 작으면 이동이 너무 적다
   - 볼록 함수(Convex Function)는 어디서 시작하더라도 경사 하강법으로 최적 값에 도달할 수 있다.
   - 비볼록 함수(Non-convex function)는 시작 위치에 따라 다른 최적 값을 찾는다. 즉, 지엽 최적값(Local Minimum)에 빠질 위험이 있다.

## 최적화 이론 Optimization Theory
 - f(x)에서 최소의 x를 구하는 것이 목적이다. f(x)는 목적 함수
 - subject to g(x)<=0, h(x)==0 [부등식 제약 조건, 등식 제약 조건] Inequality constraint, Equality constraint
 - 딥러닝에서는 대부분 제약 조건은 사용하지 않음 (미분만)

### 분석적 vs 수치적 방법
 - 분석적 방법(Analytical Method) : 함수의 모든 구간을 수식으로 알 때 사용하는 수식적인 해석 방법
 - 수치적 방법(Numerical Method) : 함수의 형태와 수식을 알지 못할 때 사용하는 계산적인 해석 방법

### 전역 vs 지역 솔루션
 - 전역 솔루션(Global Solution)은 정의역(Domain)에서 단 하나 존재한다. 지역 솔루션(Local Solution)은 어러개 일 수 있으며, 일반적으로 하나의 솔루션을 찾았을 때 local인지 global인지 확신할 수 없다.
 - 더이상 학습 안된다고 해도 그것이 전역인지 지역인지는 확신 할 수 없다.

### 안장점 Saddle Point
 - 안장점(Saddle point)는 기울기가 0이 되지만 극값이 아닌 지점을 말한다. 경사 하강법은 안장점에서 벗어나지 못한다.

### 관성(Momentum)
 - Local Minimum에 도달했으나 관성에 의해 계속 내려간다.
 - 이동벡터(Vt)를 추가로 사용하므로, 겅사 하강법 대비 2배의 메모리를 사용한다.

### 적응적 기울기 AdaGrad
 - 변수별로 학습율이 달라지게 조절하는 알고리즘
 - 기울기가 커서 학습이 많이 된 변수는 학습율을 감소시켜, 다른 변수들이 잘 학습되도록 한다.
 -  (t번째 step까지 기울기의 누적크기)가 계속 커져서 학습이 오래 진행되면 더 이상 학습이 이루어지지 않는 단점이 있다.

### AdaGrad 개선 => RMSProp
 - 합 대신 지수평균을 사용
 - 변수 간의 상대 학습율은 유지하면서 Gt가 무한정 커지지 않아 학습을 오래 할 수 있다.

### Adam
 - Adaptive moment estimation : RMSProp과 Momentum의 장점을 결합한 알고리즘
 - Adam 최적화 방법은 가장 최신의 기술(state-of-the-art)이며, 딥러닝에서 가장 많이 사용된다.
 - sota란 최고의 성능을 나타내는 최신의 기술을 나타냄

# 경사 하강법을 이용한 얕은 신경망 학습

In [22]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [32]:
EPOCHS = 1000

## 네트워크 구조 정의

### 얕은 신경망

#### 입력계층:2, 은닉계층:128(Sigmoid Activation), 출력 계층: 10(Softmax Activation)

In [24]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = tf.keras.layers.Dense(128, input_dim=2, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')
    
    def call(self, x, training=None, mask=None):
        x = self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [25]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables) #grad(loss), df(x)/dx , 스칼라를 벡터로 편미분, 그러므로 벡터가 됨
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss) # train_loss가 loss값들을 종합해줌
    train_metric(labels, predictions) # 정답과 예측값을 계산해서 평가 지표로 계산
    

## 데이터셋 생성, 전처리

In [26]:
np.random.seed(0)

pts = list()
labels = list()
center_pts = np.random.uniform(-8.0, 8.0, (10, 2))
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)

pts = np.stack(pts, axis=0).astype(np.float32) #CPU일때는 float64를 씀, GPU로 계산할 때는 float32로 동작하므로 학습데이터를 32로
labels = np.stack(labels, axis=0) #Integer이기 때문에 32로 바꿀 필요는 없음

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

## 모델 생성

In [27]:
model = MyModel()

## 손실 함수 및 최적화 알고리즘 설정

### CrossEntropy, Adam Optimizer

In [28]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 평가 지표 설정

### Accuracy

In [29]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프

In [33]:
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
    template = 'Epoch {}, Loss : {}, Accuracy :{}'
    print(template.format(epoch + 1,
                         train_loss.result(),
                         train_accuracy.result() * 100))

Epoch 1, Loss : 0.255038857460022, Accuracy :89.99833679199219
Epoch 2, Loss : 0.25503015518188477, Accuracy :89.99866485595703
Epoch 3, Loss : 0.25502046942710876, Accuracy :89.9989242553711
Epoch 4, Loss : 0.2550128996372223, Accuracy :89.99929809570312
Epoch 5, Loss : 0.2550094425678253, Accuracy :89.99962615966797
Epoch 6, Loss : 0.2550036609172821, Accuracy :89.9998550415039
Epoch 7, Loss : 0.25499585270881653, Accuracy :90.0002212524414
Epoch 8, Loss : 0.2549886405467987, Accuracy :90.00054931640625
Epoch 9, Loss : 0.25498029589653015, Accuracy :90.00092315673828
Epoch 10, Loss : 0.2549724578857422, Accuracy :90.00125885009766
Epoch 11, Loss : 0.25496241450309753, Accuracy :90.0015869140625
Epoch 12, Loss : 0.2549542188644409, Accuracy :90.0017318725586
Epoch 13, Loss : 0.2549491822719574, Accuracy :90.0020980834961
Epoch 14, Loss : 0.25493884086608887, Accuracy :90.00235748291016
Epoch 15, Loss : 0.25493165850639343, Accuracy :90.0025405883789
Epoch 16, Loss : 0.2549284100532532

Epoch 127, Loss : 0.2540573477745056, Accuracy :90.03327941894531
Epoch 128, Loss : 0.25405460596084595, Accuracy :90.03358459472656
Epoch 129, Loss : 0.25404781103134155, Accuracy :90.03382110595703
Epoch 130, Loss : 0.2540411353111267, Accuracy :90.03408813476562
Epoch 131, Loss : 0.254037469625473, Accuracy :90.03418731689453
Epoch 132, Loss : 0.2540283799171448, Accuracy :90.03421020507812
Epoch 133, Loss : 0.2540190517902374, Accuracy :90.0344467163086
Epoch 134, Loss : 0.2540103495121002, Accuracy :90.03468322753906
Epoch 135, Loss : 0.2540018558502197, Accuracy :90.03501892089844
Epoch 136, Loss : 0.2539922893047333, Accuracy :90.03532409667969
Epoch 137, Loss : 0.25398290157318115, Accuracy :90.0355224609375
Epoch 138, Loss : 0.2539748549461365, Accuracy :90.0358657836914
Epoch 139, Loss : 0.2539694607257843, Accuracy :90.03595733642578
Epoch 140, Loss : 0.2539632320404053, Accuracy :90.03608703613281
Epoch 141, Loss : 0.2539561092853546, Accuracy :90.03643035888672
Epoch 142, 

Epoch 257, Loss : 0.2530580759048462, Accuracy :90.06565856933594
Epoch 258, Loss : 0.25304970145225525, Accuracy :90.06591033935547
Epoch 259, Loss : 0.2530438303947449, Accuracy :90.0661849975586
Epoch 260, Loss : 0.25303444266319275, Accuracy :90.06620025634766
Epoch 261, Loss : 0.25302544236183167, Accuracy :90.06648254394531
Epoch 262, Loss : 0.25301656126976013, Accuracy :90.06673431396484
Epoch 263, Loss : 0.2530103027820587, Accuracy :90.06700897216797
Epoch 264, Loss : 0.25300389528274536, Accuracy :90.06732940673828
Epoch 265, Loss : 0.25299495458602905, Accuracy :90.06777954101562
Epoch 266, Loss : 0.25298523902893066, Accuracy :90.06806182861328
Epoch 267, Loss : 0.25297826528549194, Accuracy :90.06830596923828
Epoch 268, Loss : 0.2529703676700592, Accuracy :90.06842041015625
Epoch 269, Loss : 0.25296705961227417, Accuracy :90.06866455078125
Epoch 270, Loss : 0.2529587149620056, Accuracy :90.06877899169922
Epoch 271, Loss : 0.2529544532299042, Accuracy :90.06912231445312
Ep

Epoch 381, Loss : 0.2521333396434784, Accuracy :90.09601593017578
Epoch 382, Loss : 0.25212961435317993, Accuracy :90.09605407714844
Epoch 383, Loss : 0.2521226108074188, Accuracy :90.09611511230469
Epoch 384, Loss : 0.25211474299430847, Accuracy :90.09634399414062
Epoch 385, Loss : 0.25210705399513245, Accuracy :90.09640502929688
Epoch 386, Loss : 0.25209856033325195, Accuracy :90.0967025756836
Epoch 387, Loss : 0.25209200382232666, Accuracy :90.09689331054688
Epoch 388, Loss : 0.2520837187767029, Accuracy :90.09709167480469
Epoch 389, Loss : 0.2520759105682373, Accuracy :90.09735870361328
Epoch 390, Loss : 0.2520700991153717, Accuracy :90.09745025634766
Epoch 391, Loss : 0.25206249952316284, Accuracy :90.0976791381836
Epoch 392, Loss : 0.2520533502101898, Accuracy :90.0978775024414
Epoch 393, Loss : 0.25204578042030334, Accuracy :90.09806823730469
Epoch 394, Loss : 0.2520400285720825, Accuracy :90.09839630126953
Epoch 395, Loss : 0.25203242897987366, Accuracy :90.09855651855469
Epoch

Epoch 512, Loss : 0.25122007727622986, Accuracy :90.12434387207031
Epoch 513, Loss : 0.25121182203292847, Accuracy :90.12464904785156
Epoch 514, Loss : 0.2512066960334778, Accuracy :90.12476348876953
Epoch 515, Loss : 0.25119879841804504, Accuracy :90.12494659423828
Epoch 516, Loss : 0.251191645860672, Accuracy :90.125244140625
Epoch 517, Loss : 0.2511850893497467, Accuracy :90.12551879882812
Epoch 518, Loss : 0.25117847323417664, Accuracy :90.12579345703125
Epoch 519, Loss : 0.2511707544326782, Accuracy :90.12606048583984
Epoch 520, Loss : 0.251163512468338, Accuracy :90.12621307373047
Epoch 521, Loss : 0.2511581480503082, Accuracy :90.12641906738281
Epoch 522, Loss : 0.25115320086479187, Accuracy :90.12665557861328
Epoch 523, Loss : 0.25114497542381287, Accuracy :90.12686920166016
Epoch 524, Loss : 0.25113677978515625, Accuracy :90.12714385986328
Epoch 525, Loss : 0.25113019347190857, Accuracy :90.12738037109375
Epoch 526, Loss : 0.25112220644950867, Accuracy :90.1275863647461
Epoch 

Epoch 643, Loss : 0.2503342032432556, Accuracy :90.15194702148438
Epoch 644, Loss : 0.2503277063369751, Accuracy :90.1520767211914
Epoch 645, Loss : 0.25032150745391846, Accuracy :90.1522445678711
Epoch 646, Loss : 0.25031858682632446, Accuracy :90.15243530273438
Epoch 647, Loss : 0.2503111660480499, Accuracy :90.15272521972656
Epoch 648, Loss : 0.25030380487442017, Accuracy :90.15288543701172
Epoch 649, Loss : 0.2502957880496979, Accuracy :90.15310668945312
Epoch 650, Loss : 0.2502880394458771, Accuracy :90.15345001220703
Epoch 651, Loss : 0.2502845227718353, Accuracy :90.15367889404297
Epoch 652, Loss : 0.2502771317958832, Accuracy :90.15392303466797
Epoch 653, Loss : 0.2502704858779907, Accuracy :90.15414428710938
Epoch 654, Loss : 0.2502628266811371, Accuracy :90.15437316894531
Epoch 655, Loss : 0.2502559423446655, Accuracy :90.15465545654297
Epoch 656, Loss : 0.2502477765083313, Accuracy :90.15484619140625
Epoch 657, Loss : 0.2502467930316925, Accuracy :90.1551284790039
Epoch 658,

Epoch 775, Loss : 0.2495046705007553, Accuracy :90.17781829833984
Epoch 776, Loss : 0.24949876964092255, Accuracy :90.17805480957031
Epoch 777, Loss : 0.24949140846729279, Accuracy :90.17823028564453
Epoch 778, Loss : 0.24948345124721527, Accuracy :90.17838287353516
Epoch 779, Loss : 0.24947725236415863, Accuracy :90.17847442626953
Epoch 780, Loss : 0.249471053481102, Accuracy :90.17859649658203
Epoch 781, Loss : 0.24946388602256775, Accuracy :90.17874908447266
Epoch 782, Loss : 0.2494577169418335, Accuracy :90.17892456054688
Epoch 783, Loss : 0.24945050477981567, Accuracy :90.17904663085938
Epoch 784, Loss : 0.24944256246089935, Accuracy :90.1793441772461
Epoch 785, Loss : 0.2494354099035263, Accuracy :90.17945861816406
Epoch 786, Loss : 0.24942876398563385, Accuracy :90.17958068847656
Epoch 787, Loss : 0.24942582845687866, Accuracy :90.17967224121094
Epoch 788, Loss : 0.24941882491111755, Accuracy :90.17979431152344
Epoch 789, Loss : 0.2494116872549057, Accuracy :90.1800537109375
Epo

Epoch 900, Loss : 0.24872931838035583, Accuracy :90.19898986816406
Epoch 901, Loss : 0.24872401356697083, Accuracy :90.19915771484375
Epoch 902, Loss : 0.2487209290266037, Accuracy :90.19929504394531
Epoch 903, Loss : 0.2487156093120575, Accuracy :90.19949340820312
Epoch 904, Loss : 0.24870949983596802, Accuracy :90.19966125488281
Epoch 905, Loss : 0.2487025409936905, Accuracy :90.19991302490234
Epoch 906, Loss : 0.24869565665721893, Accuracy :90.2000732421875
Epoch 907, Loss : 0.24869254231452942, Accuracy :90.2000732421875
Epoch 908, Loss : 0.24868625402450562, Accuracy :90.20027160644531
Epoch 909, Loss : 0.24868059158325195, Accuracy :90.20035552978516
Epoch 910, Loss : 0.24867385625839233, Accuracy :90.20040893554688
Epoch 911, Loss : 0.24866673350334167, Accuracy :90.20063018798828
Epoch 912, Loss : 0.24866002798080444, Accuracy :90.20079040527344
Epoch 913, Loss : 0.24865588545799255, Accuracy :90.20093536376953
Epoch 914, Loss : 0.248650461435318, Accuracy :90.20104217529297
Ep

## 데이터셋 및 학습 파라미터 저장

In [36]:
np.savez_compressed('ch2_dataset.npz', inputs=pts, labels=labels)

W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ch2_parameters.npz',
                   W_h=W_h,
                   b_h=b_h,
                   W_o=W_o,
                   b_o=b_o)